In [3]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [4]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_DF = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
lending_DF.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [5]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_DF['loan_status']

# Separate the X variable, the features
X = lending_DF.drop(columns=['loan_status'])

In [6]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [7]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [8]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [9]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [10]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model_assign = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model_fit = lr_model_assign.fit(x_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [11]:
# Make a prediction using the testing data
test_predictions = lr_model_assign.predict(x_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [12]:
# Print the balanced_accuracy score of the model
accuracy_score = balanced_accuracy_score(y_test, test_predictions)
print(accuracy_score)

0.9520479254722232


In [13]:
# Generate a confusion matrix for the model
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[18663   102]
 [   56   563]]


In [14]:
# Print the classification report for the model
test_report = classification_report(y_test, test_predictions)

# Print the training classification report
print(test_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts our healthy loans fantastically with a precision of 100% and has a recall of 99% meaning that the model predicts heathy loan labels very accurately. Our high-risk have a precision of 85% and a recall of 91% which are still high accuracy values, but not as good as our healthy loans.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [15]:
pip install -q imblearn

In [16]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
over_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
x_2, y_2 = over_model.fit_resample(x_train, y_train)
# using 'imbalanced-learn.org' found the functions and documentation for RandomOverSampler

In [17]:
# Count the distinct values of the resampled labels data
y_2.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [18]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier.fit(x_2, y_2)

# Make a prediction using the testing data
predictions = classifier.predict(x_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).tail()

,Prediction,Actual
45639,0,0
11301,0,0
51614,0,0
4598,0,0
2793,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [19]:
# Print the balanced_accuracy score of the model
accuracy_score_2 = balanced_accuracy_score(y_test, predictions)
print(accuracy_score_2)

0.9936781215845847


In [20]:
# Generate a confusion matrix for the model
matrix_2 = confusion_matrix(y_test, predictions)
print(matrix_2)

[[18649   116]
 [    4   615]]


In [21]:
# Print the classification report for the model
report_2 = classification_report(y_test, predictions)

# Print the training classification report
print(report_2)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The new model predicts our healthy loans very well with a precision of 100% and has a recall of 99% meaning that the model predicts heathy loan labels very accurately. Our high-risk loans have a lower precision at 84% but a higher recall at 99%. So we have a slightly higher amount of false positives in this model but a much much lower amount of false negatives than in the first model. Hence, I would much rather use our RandomOverSampler regression model. We also see a much higher accuracy score in the second model than in the first.